# Introduction à l'apprentissage automatique: TP additionnel - <font color=red> CORRECTION </font>

<br>

### Mélanges de gaussiennes et jeu de données "Old Faithful", prédiction par Gaussian Mixture Regression

<br>
On va étudier un jeu de données relatif au geyser "Old Faithful" dans le parc du Yellowstone aux Etats-Unis.

Chaque observation est constituée de deux caractéristiques: la durée d'une éruption et la durée avant l'éruption suivante.


Les données sont décrites ici: 
http://www.stat.cmu.edu/~larry/all-of-statistics/=data/faithful.dat <br>
et doivent être téléchargées sur Arche ou [sur la page web du cours](https://members.loria.fr/FSur/enseignement/apprauto/old_faithful.txt).

<br>
On commence par charger les bibliothèques qui nous seront utiles:



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from sklearn import mixture

%matplotlib notebook 

puis on charge les données du fichier `old_faithful.txt`:

In [ ]:
# attention: le notebook doit être ouvert dans le même répertoire que le fichier old_faithful.txt
data=np.loadtxt('old_faithful.txt')
print(data)  # pour vérifier que les données sont bien chargées

On commence par représenter les données par un nuage de point.


In [ ]:
plt.figure(figsize=[12,10]);
plt.scatter(data[:,1], data[:,2]);
plt.grid()
plt.xlabel('durée éruption')
plt.ylabel('temps avant la prochaine éruption')
plt.title('Old Faithful');

## Modélisation d'une densité de probabilité par mélange de gaussiennes

Les observations sont des couples $(x,y)$ où $x$ est la durée d'éruption et $y$ le temps avant la prochaine éruption. Nous allons modéliser la loi jointe du couple $(x,y)$ à l'aide d'un mélange de gaussiennes.

__Question 1__. Observez l'évolution du critère BIC dans l'ajustement d'un mélange de gaussiennes à $M$ composantes, pour $M$ variant entre 1 et 10 (inspirez-vous du code ci-dessous, et utilisez la [documentation](http://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html)). Vous afficherez les lignes d'iso-valeur du logarithme de la densité de probabilité estimée pour chaque valeur de $M$ en utilisant la fonction `show_mixture` ci-dessous (inspirée du code de la [documentation scikit-learn](http://scikit-learn.org/stable/auto_examples/mixture/plot_gmm_pdf.html)).

Constatez que l'évolution du critère BIC semble justifier que ce geyser présente deux "modes de fonctionnement". 

<br>

_Remarque 1_ : $M$ est appelé un _hyperparamètre_ du modèle car il doit être fixé par l'utilisateur, à l'aide d'indicateurs comme BIC ici. Les paramètres du modèle sont les poids, moyennes, et matrices de covariance: ils sont estimés à partir des données par la fonction `fit` ci-dessous (qui, ici, met en oeuvre l'algorithme EM).

_Remarque 2_ : par définition, la log-vraisemblance calculée à partir d'une observation (ce qui est retourné par `score_samples` ci-dessous) est le logarithme de la valeur de la densité de probabilité en cette observation. 

In [ ]:
def show_mixture(mixture,X_train):
    # mixture: objet GaussianMixture ajusté à des données
    # X_train: pour le scatter-plot, observations sous forme d'un tableau (observations en lignes, caractéristiques en colonnes)
    x = np.linspace(1, 5.5)
    y = np.linspace(20., 120.)
    X, Y = np.meshgrid(x, y)
    XX = np.array([X.ravel(), Y.ravel()]).T
    Z = -mixture.score_samples(XX)  # score_samples est le log de la densité (log-vraisemblance) en chaque point  
    Z = Z.reshape(X.shape)
    plt.figure(figsize=[8,6])
    CS = plt.contour(X, Y, Z, norm=LogNorm(vmin=1.0, vmax=100.0),
                    levels=np.logspace(0, 3, 10))   # lignes d'iso-valeur
    CB = plt.colorbar(CS, shrink=0.8, extend='both')
    plt.scatter(X_train[:, 0], X_train[:, 1])  # nuage des observations
    plt.scatter(mixture.means_[:,0],mixture.means_[:,1],c='red')  # moyennes des composantes en rouge
    plt.title('Level lines of negative log-likelihood predicted by a GMM, K='+str(mixture.n_components))
    plt.axis('tight')
    plt.show();

In [ ]:
# je ne demande que le cas "full", mais je mets le code pour essayer les autres modèles
# (voir la documentation de GaussianMixture pour la signification des types de covariance)

print("Full\n")
BIC_full=[]
for K in range(1,11):
    GMM=mixture.GaussianMixture(n_components=K,covariance_type='full')
    GMM.fit(data[:,1:3])
    BIC_full.append(GMM.bic(data[:,1:3]))
    print('K: %d  BIC=%.2f' %(K,BIC_full[-1]))
    show_mixture(GMM,data[:,1:3])

plt.figure()
plt.plot(range(1,11),BIC_full)
plt.legend(("full"))          
plt.xlabel('K')
plt.ylabel('BIC')
plt.grid()
plt.show();

print("Tied\n")
BIC_tied=[]
for K in range(1,11):
    GMM=mixture.GaussianMixture(n_components=K,covariance_type='tied')
    GMM.fit(data[:,1:3])   
    BIC_tied.append(GMM.bic(data[:,1:3]))
    print('K: %d  BIC=%.2f' %(K,BIC_tied[-1]))
    show_mixture(GMM,data[:,1:3])
    
print("Diag\n")
BIC_diag=[]
for K in range(1,11):
    GMM=mixture.GaussianMixture(n_components=K,covariance_type='diag')
    GMM.fit(data[:,1:3])   
    BIC_diag.append(GMM.bic(data[:,1:3]))
    print('K: %d  BIC=%.2f' %(K,BIC_diag[-1]))
    show_mixture(GMM,data[:,1:3])
          
print("Spherical\n")
BIC_sphe=[]
for K in range(1,11):
    GMM=mixture.GaussianMixture(n_components=K,covariance_type='spherical')
    GMM.fit(data[:,1:3])   
    BIC_sphe.append(GMM.bic(data[:,1:3]))
    print('K: %d  BIC=%.2f' %(K,BIC_sphe[-1]))
    show_mixture(GMM,data[:,1:3])
          
plt.figure()
plt.plot(range(1,11),BIC_full)
plt.plot(range(1,11),BIC_tied)
plt.plot(range(1,11),BIC_diag)
plt.plot(range(1,11),BIC_sphe)
plt.legend(("full","tied","diag","spherical"))          
plt.xlabel('K')
plt.ylabel('BIC')
plt.grid()

<font color=red> 
    
Le dernier graphe nous montre que $K=2$ est bien minimum du critère BIC
    
</font>

__Question 2__. Ecrivez les paramètres (poids, moyennes, matrices de covariance) du mélange de gaussienne optimal au sens de BIC.

In [ ]:
GMM = mixture.GaussianMixture(n_components=2)
GMM.fit(data[:,1:3])  
print("BIC: %.3f" %GMM.bic(data[:,1:3])) 

print("\nweights:")  # on vérifie que la somme des poids est 1
print(GMM.weights_)

print("\nmeans:")
print(GMM.means_)

print("\ncovariances:")
print(GMM.covariances_)


<font color=red>
    
Densité estimée:
$$ p(x,y) = \pi_1 {\cal G}_{\mu_1,\Sigma_1} (x,y) + \pi_2 {\cal G}_{\mu_2,\Sigma_2} (x,y)$$  
avec les notations de la section 4.2.2.2 du polycopié.

Les moyennes sont les points rouges dans les graphiques de `show_mixture`. Les moyennes et matrices de covariance ont des grandes valeurs selon $y$ car cette variable varie dans un intervalle beaucoup plus grand que $x$.
    
</font>

__Question 3__. La "méthode du coude" de $K$-Means permet-elle de justifier un classification en deux groupes ?

In [ ]:
# voir le code de l'exercice 1 
from sklearn import cluster

inertie=np.zeros((10))
for K in range(1,11):
    clustering=cluster.KMeans(n_clusters=K)
    clustering.fit(data[:,1:3])
    inertie[K-1]=clustering.inertia_
plt.figure();
plt.plot(np.arange(1,11),inertie);
plt.xlabel("K");
plt.ylabel("inertie");
plt.title("elbow plot");

clustering=cluster.KMeans(n_clusters=2)
clustering.fit(data[:,1:3])
plt.figure()
plt.scatter(data[:, 1], data[:, 2], s=10,c=clustering.labels_)
plt.title('Kmeans, n_cluster=2');

<font color=red>
    
Oui: on voit un inflexion assez franche dans le elbow plot pour $K=2$.
    
Ici, les groupes peuvent être identifiés avec $K$-means, on peut bien utiliser la méthode du coude.

</font>

## Illustration de la théorie développée au chapitre 4, section 4.1.1

## Prédiction du temps avant la prochaine éruption par Gaussian mixture regression

Nous allons à présent prédire le temps d'attente avant la prochaine éruption en fonction de la durée d'une éruption.

Si $x$ désigne la durée d'une éruption et $y$ le temps avant la prochaine éruption, nous venons de modéliser la densité $p(x,y)$ de la loi de probabilité jointe de $x$ et $y$ comme un mélange de deux gaussiennes.

Nous verrons lors de la prochaine séance (polycopié chapitre 4, section 4.1.1) que la meilleure prédiction (en un sens précis) de $y$ en fonction de $x$ est donnée par l'espérance conditionnelle suivante:
$$ h(x) = E(y|x) = \int_y y p(y|x) dy$$
Il s'agit de l'espérance de $y$ connaissant $x$. Autrement dit, il s'agit de la valeur moyenne de $y$ à $x$ fixé.

Par définition,
$$p(y|x) = \frac{p(x,y)}{p(x)}$$
et la densité marginale $p(x)$ est déterminée par:
$$ p(x) = \int_y p(x,y) dy$$
Malheureusement, `GaussianMixture` de scikit-learn n'implémente pas ces calculs. Nous allons donc déterminer l'espérance conditionnelle $h(x)$ par intégration numérique, à partir du mélange de gaussiennes $p(x,y)$.

C'est ce que fait la fonction `GMM_regression` dans la cellule suivante: ses arguments sont 1) un objet `GaussianMixture` représentant un mélange de Gaussiennes, et 2) un tableau de données $x$; elle retourne le tableau des valeurs prédites pour les valeurs de $x$ fournies.

In [ ]:
def GMM_regression(GMM,x):
    y = np.linspace(20., 120.,num=1001)  # y discrétisé entre 20 et 120 par pas de 0.1 (1001 valeurs)
    X, Y = np.meshgrid(x, y)
    XX = np.array([X.ravel(), Y.ravel()]).T
    p = np.exp(GMM.score_samples(XX))   # score_samples fournit la log-vraisemblance d'une observation, p est donc bien la valeur de la densité  
    p = p.reshape(X.shape)  # on construit ainsi la carte des p(x,y)
    condexp = np.sum(Y*p,axis=0)/np.sum(p,axis=0)  # espérance conditionnelle obtenue par intégration numérique (ici, le première axe correspond à y)
    return condexp

__Question 4__. Complétez le code suivant de manière à tracer le graphique de la prédiction du temps entre éruptions en fonction de la durée d'éruption.

Comparez votre prédiction avec ce qui est proposé sur __[cette page](https://www.nps.gov/teachers/classrooms/predict-old-faithful.htm)__ du National Park Service. (nos données sont un peu ancienne, le "fonctionnement" du geyser évolue au fil des années)

In [ ]:
GMM=mixture.GaussianMixture(n_components=2,covariance_type='full')
GMM.fit(data[:,1:3])
x = np.linspace(1, 5.5)

condexp = GMM_regression(GMM,x)

plt.figure(figsize=[12,10])
plt.scatter(data[:,1], data[:,2]);
plt.plot(x,condexp,'r')
plt.grid()
plt.xlabel('durée éruption')
plt.ylabel('temps entre éruptions')
plt.title('Old Faithful')
plt.legend(("GMM regression",'data'))
plt.show();

<font color=red>
    
Notre prédiction identifie bien un changement de régime à partir de $x\simeq 3$ minutes.
    
L'espérance conditionnelle $E(y|x)$ dans le cas d'une loi jointe gaussienne correspond exactement à la régression linéaire (voir aussi le cours d'analyse de données). Ceci explique l'aspect de la courbe de régression: pour x franchement inférieur à 3 ou franchement supérieur à 3, la loi jointe $p(x,y)$ est dominée par l'une ou l'autre des deux gaussiennes, l'autre ayant une contribution très faible. Dans chaque cas, la courbe de régression est donc très proche d'un segment de droite.
    
Cette __[vidéo](https://www.nps.gov/yell/learn/photosmultimedia/indepth-predictingoldfaithful.htm)__ suggère que d'autres éléments sont pris en compte dans la prédiction par les rangers du Yellowstone Park.
    
</font>

__Question 5__. Tracez la droite de régression linéaire superposée à l'estimation par espérance conditionnelle dans laquelle la loi jointe $p(x,y)$ est supposée gaussienne (il suffit de faire `n_components=1` dans `GaussianMixture`), et constatez que les prédictions sont identiques dans les deux cas. La raison est expliquée dans la section consacrée à la régression dans le polycopié.

In [ ]:
import sklearn.linear_model as lm
GMM=mixture.GaussianMixture(n_components=1,covariance_type='full')
GMM.fit(data[:,1:3])
x = np.linspace(1, 5.5)

linear_reg=lm.LinearRegression()
linear_reg.fit(data[:,1].reshape(-1, 1),data[:,2].reshape(-1, 1))

condexp = GMM_regression(GMM,x)
plt.figure(figsize=[12,10])

plt.scatter(data[:,1], data[:,2]);
x = np.linspace(1, 5.5)
plt.plot(x,condexp,'r')
plt.plot(x,linear_reg.predict(x.reshape(-1,1)),'g*')
plt.grid()
plt.xlabel('durée éruption')
plt.ylabel('temps entre éruptions')
plt.title('Old Faithful')
plt.legend(('GMM regression','lm','data'))
plt.show();

<font color=red>
    
Par contre l'estimation des paramètres ne se fait pas de la même manière dans les deux approches: EM pour GMM, équations normales pour régression linéaire. Cela explique les légères différences.
   
</font>